In [0]:
import keras
from keras.layers import LeakyReLU, Conv2D, Conv2DTranspose, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Reshape, Activation
from keras.models import Model,Sequential
from keras.optimizers import Adam, RMSprop

import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
plt.ion()
from os import walk
import matplotlib.image as mpimg

Using TensorFlow backend.


In [0]:
batch_size = 256
epochs  = 50000
noise_size = 100

In [0]:
#choose the correct zip to extract
!unzip pokemon_small.zip
#!unzip augmented.zip

Archive:  pokemon_small.zip
 extracting: pokemon_small/1.png     
 extracting: pokemon_small/10.png    
 extracting: pokemon_small/100.png   
 extracting: pokemon_small/101.png   
 extracting: pokemon_small/102.png   
 extracting: pokemon_small/103.png   
 extracting: pokemon_small/104.png   
 extracting: pokemon_small/105.png   
 extracting: pokemon_small/106.png   
 extracting: pokemon_small/107.png   
 extracting: pokemon_small/108.png   
 extracting: pokemon_small/109.png   
 extracting: pokemon_small/11.png    
 extracting: pokemon_small/110.png   
 extracting: pokemon_small/111.png   
 extracting: pokemon_small/112.png   
 extracting: pokemon_small/113.png   
 extracting: pokemon_small/114.png   
 extracting: pokemon_small/115.png   
 extracting: pokemon_small/115-mega.png  
 extracting: pokemon_small/116.png   
 extracting: pokemon_small/117.png   
  inflating: pokemon_small/118.png   
 extracting: pokemon_small/119.png   
 extracting: pokemon_small/12.png    
 extracting: pokem

In [0]:
#------------------------------------------------------------------------------------
# Define the generator - take noise and convert them to images
#------------------------------------------------------------------------------------
def generator():
    generator=Sequential()
    
    # output: 8*8*512
    generator.add(Dense(8*8*512, input_shape=(noise_size,)))
    generator.add(BatchNormalization(momentum=0.9))
    generator.add(LeakyReLU(alpha=0.2))

    #output: (8,8,512)
    generator.add(Reshape((8, 8, 512)))
    generator.add(Dropout(0.5)) 
   
    # output: (8,8,256)
    generator.add(Conv2DTranspose(256, kernel_size=(5,5), padding='same'))
    generator.add(BatchNormalization(momentum=0.9))
    generator.add(LeakyReLU(alpha=0.2))

    # output: (16,16,128)
    generator.add(Conv2DTranspose(128, strides=(2, 2), kernel_size=(5,5), padding='same'))
    generator.add(BatchNormalization(momentum=0.9))
    generator.add(LeakyReLU(alpha=0.2))
    
    # output: (32,32,64)
    generator.add(Conv2DTranspose(64, strides=(2, 2), kernel_size=(5,5), padding='same'))
    generator.add(BatchNormalization(momentum=0.9))
    generator.add(LeakyReLU(alpha=0.2))
    
    # output: (64,64,3)
    generator.add(Conv2DTranspose(3, strides=(2, 2), kernel_size=(5,5), padding='same', activation='sigmoid'))
    
    return generator

g=generator()
g.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32768)             3309568   
_________________________________________________________________
batch_normalization_1 (Batch (None, 32768)             131072    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 512)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 512)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 8, 8, 256)         32770

noise = tf.random.normal([1, 200])
print(g(noise)[0].shape)
generated = g(noise)[0]
print(generated[0,1,0])

generated=np.array(generated).reshape((64,64,3))
print(generated.dtype)
print(generated.shape)
plt.imshow(generated[0])

In [0]:
#------------------------------------------------------------------------------------
# New loss function based on the wasserstein image
#------------------------------------------------------------------------------------
from keras import backend as K
def wasserstein_loss(y_true, y_pred):
	return K.mean(y_true * y_pred)

In [0]:
#------------------------------------------------------------------------------------
# Weight clipping constraint used by the critic model
#------------------------------------------------------------------------------------
from keras.constraints import Constraint

class WeightClip(Constraint):

    def __init__(self, c=2):
        self.c = c

    def __call__(self, p):
        return K.clip(p, -self.c, self.c)

    def get_config(self):
        return {'name': self.__class__.__name__,
                'c': self.c}

In [0]:
#------------------------------------------------------------------------------------
# Define the critic
#------------------------------------------------------------------------------------
def critic():
    model = Sequential()
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[64, 64, 3],
                     kernel_constraint=WeightClip(0.1)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))

    model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same',
                     kernel_constraint=WeightClip(0.1)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(256, (5, 5), strides=(2, 2), padding='same',
                     kernel_constraint=WeightClip(0.1)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))

    model.add(Flatten())
    
    model.add(Dense(1, activation='linear',
                     W_constraint=WeightClip(0.1)))

    return model

c=critic()
c.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 64)        4864      
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 128)       204928    
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 8, 8, 256)        

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="linear", kernel_constraint=<__main__....)`


In [0]:
def critic_model():
    optimizer = RMSprop(lr=0.00005, clipvalue=1.0, decay=6e-8)
    model = Sequential()
    model.add(c)
    model.compile(loss=wasserstein_loss, optimizer=optimizer, metrics=['accuracy'])
    return model

cr = critic_model()
cr.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_8 (Sequential)    (None, 1)                 1045633   
Total params: 1,045,633
Trainable params: 1,045,633
Non-trainable params: 0
_________________________________________________________________


In [0]:
def adversarial_model():
    # ezt csak én találtam ide ki, nem tudom így úgy működik e ahogy elvárnánk
    for layer in c.layers:
        layer.trainable=False
        
    optimizer = RMSprop(lr=0.0005, clipvalue=1.0, decay=3e-8)
    model = Sequential()
    model.add(g)
    #model.add(d)
    model.add(c)
    model.compile(loss=wasserstein_loss, optimizer=optimizer, metrics=['accuracy'])
    return model

am = adversarial_model()
am.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 64, 64, 3)         7748483   
_________________________________________________________________
sequential_8 (Sequential)    (None, 1)                 1045633   
Total params: 8,794,116
Trainable params: 7,682,051
Non-trainable params: 1,112,065
_________________________________________________________________


In [0]:
#----------------------------------------------------------------------------
# Read the dataset
#----------------------------------------------------------------------------
imgs = []

f = []
for (dirpath, dirnames, filenames) in walk('pokemon_small'):
    f.extend(filenames)

for src in f:
    #read PNG images and convert them to RGB tensors
    img = mpimg.imread('pokemon_small/' + src)[:,:,0:3]
    imgs.append(img)
    
imgs = np.array(imgs)

In [0]:
def train():
  trainA, trainD = True, True
  for i in range(epochs):  
    #train the critic on every fifth epoch  
    if(epochs%5==0):
      trainA = True
    else:
      trainA =False

  # same training as the DCGAN
    if (trainD):
      imgs_train = imgs[np.random.randint(0,imgs.shape[0], size=batch_size), :, :, :]
      noise = np.random.normal(0, 1.0, size=[batch_size, noise_size])
      imgs_fake = g.predict(noise)
      x = np.concatenate((imgs_train, imgs_fake))
      y = np.ones([2*batch_size, 1])
      # the generated images are labeled -1 for the loss
      y[batch_size:, :] = -1
      d_loss = cr.train_on_batch(x, y)

    if (trainA):
      y = np.ones([batch_size, 1])
      noise = np.random.normal(0, 1.0, size=[batch_size, noise_size])
      a_loss = am.train_on_batch(noise, y)

    if (i % 50 == 0):
      log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
      log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
      print(log_mesg)

    if (i % 250 == 0):
      plot_images(fake = True)   

In [0]:
def plot_images(save2file=False, fake=True, samples=16, noise=None, step=0):
        filename = 'pokemon.png'
        if fake:
            if noise is None:
                noise = np.random.normal(0, 1.0, size=[samples, noise_size])
            else:
                filename = "pokemon_%d.png" % step
            images = g.predict(noise)
        else:
            i = np.random.randint(0, imgs.shape[0], samples)
            images = imgs[i, :, :, :]
        print(images.shape)

        plt.figure(figsize=(10,10))
        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [64, 64, 3])
            plt.imshow(image)
            plt.axis('off')
        plt.tight_layout()
        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()

In [0]:
train()